nextwordd.streamlit.app

In [ ]:
import torch
import torch.nn.functional as F
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt # for making figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
from pprint import pprint

In [ ]:
import pandas as pd
import requests
import re
from pprint import pprint

# Function to fetch and clean data from the URL
def fetch_and_clean_data(url):
    response = requests.get(url)
    response.raise_for_status()
    lines = response.text.splitlines()

    cleaned_lines = []
    for line in lines:
        cleaned_line = line.strip().lower()
        if cleaned_line:
            cleaned_line = re.sub(r'[^a-zA-Z0-9 \.]', '', cleaned_line)
            cleaned_lines.append(cleaned_line)

    return cleaned_lines

# Load the dataset directly from the URL
url = 'https://www.gutenberg.org/files/1661/1661-0.txt'
cleaned_lines = fetch_and_clean_data(url)

# Convert to a single text
text = ' '.join(cleaned_lines)

# Split the text into words
words = text.split()  # Split the text into words

# Build the vocabulary of words and mappings to/from integers
unique_words = sorted(set(words))
stoi = {s: i + 1 for i, s in enumerate(unique_words)}
stoi['.'] = 0
itos = {i: s for s, i in stoi.items()}

def create_sequences(words, stoi, seq_length=5):
    sequences = []
    for i in range(len(words) - seq_length):
        input_seq = words[i:i + seq_length]
        target_word = words[i + seq_length]
        input_indices = [stoi[word] for word in input_seq]
        sequences.append((input_indices, stoi[target_word]))
    return sequences

# Create training sequences
sequences = create_sequences(words, stoi)

# Print the first few sequences
for seq in sequences[:5]:
    print(f"Input: {seq[0]}, Target: {seq[1]}")

Input: [9239, 7171, 4299, 3027, 6380], Target: 9239
Input: [7171, 4299, 3027, 6380, 9239], Target: 288
Input: [4299, 3027, 6380, 9239, 288], Target: 6380
Input: [3027, 6380, 9239, 288, 6380], Target: 8189
Input: [6380, 9239, 288, 6380, 8189], Target: 4587


In [ ]:
import torch

# Define block size
block_size = 5  # Context length: how many words do we take to predict the next one?
X, Y = [], []

# Create sequences for next-word prediction
for sentence in cleaned_lines:
    word_list = sentence.split()
    context = [0] * block_size

    for word in word_list + ['.']:
        ix = stoi.get(word, stoi['.'])
        X.append(context.copy())
        Y.append(ix)

        print(' '.join(itos[i] for i in context), '--->', itos[ix])

        context = context[1:] + [ix]

# Move data to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X = torch.tensor(X).to(device)
Y = torch.tensor(Y).to(device)

# Print the shapes of X and Y
print(f'Shape of X: {X.shape}, Shape of Y: {Y.shape}')


Streaming output truncated to the last 5000 lines.
what on earth has that ---> to
on earth has that to ---> do
earth has that to do ---> with
has that to do with ---> it
that to do with it ---> i
to do with it i ---> ejaculated.
do with it i ejaculated. ---> .
. . . . . ---> my
. . . . my ---> dear
. . . my dear ---> watson
. . my dear watson ---> you
. my dear watson you ---> as
my dear watson you as ---> a
dear watson you as a ---> medical
watson you as a medical ---> man
you as a medical man ---> are
as a medical man are ---> continually
a medical man are continually ---> gaining
medical man are continually gaining ---> light
man are continually gaining light ---> as
are continually gaining light as ---> .
. . . . . ---> to
. . . . to ---> the
. . . to the ---> tendencies
. . to the tendencies ---> of
. to the tendencies of ---> a
to the tendencies of a ---> child
the tendencies of a child ---> by
tendencies of a child by ---> the
of a child by the ---> study
a child by the study --

In [ ]:
emb_dim = 32
emb = torch.nn.Embedding(len(stoi), emb_dim)
import torch
import torch.nn as nn

class NextWord(nn.Module):
    def __init__(self, block_size, vocab_size, emb_dim, hidden_size=1024):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.lin1 = nn.Linear(block_size * emb_dim, hidden_size)
        self.lin2 = nn.Linear(hidden_size, vocab_size)

    def forward(self, x):
        x = self.emb(x)
        x = x.view(x.shape[0], -1)
        x = torch.tanh(self.lin1(x))  # Use tanh or relu activation
        x = self.lin2(x)
        return x


In [ ]:
# Generate names from untrained model
model = NextWord(block_size, len(stoi), emb_dim, 1024).to(device)
model = torch.compile(model)

g = torch.Generator()
g.manual_seed(4000002)

def generate_name(model, itos, stoi, block_size, max_len=10):
    context = [0] * block_size
    name = ''
    for i in range(max_len):
        x = torch.tensor(context).view(1, -1).to(device)
        y_pred = model(x)
        ix = torch.distributions.categorical.Categorical(logits=y_pred).sample().item()
        word = itos[ix]

        if word == '.':
            break

        name += (' ' + word) if name else word
        context = context[1:] + [ix]
    return name

# Generate and print names
for i in range(10):
    print(generate_name(model, itos, stoi, block_size))


success. 2000 indulge flicking level witness warm orgies wellnigh briefly
constructed x. incriminate woke bind expression purposes yards thither elderly
statement indulgently. tortured whipcord mask. explaining. individuality disappearance honest earliest
fixed amplifying ear. warsawyes aunts imply. ebook second. incisive galeand
jewels landscape. enthusiastic sink remorseless concerts present being inscrutable everybody
page supper. duncoloured conception die inhabited arteries firemen mess 3rd.
hadnt softened san disregarding spring greengrocer settled array case firmly.
endeavouring ears. lefthanded forefinger witnesses grievous searched loud used crushing
reopening blurs gasped. watered circles oakshott 3rd. haste fourandtwenty. forgive
wellnurtured remarked. clouds regretted covered scissors lumberroom talk. aware untimely


In [ ]:
loss_fn = nn.CrossEntropyLoss()
opt = torch.optim.AdamW(model.parameters(), lr=0.01)

import time

# Mini-batch training
batch_size = 15000  # Adjust batch size based on available memory
print_every = 50  # Print every few epochs for better visibility
elapsed_time = []

for epoch in range(500):
    start_time = time.time()
    for i in range(0, X.shape[0], batch_size):
        x = X[i:i+batch_size]
        y = Y[i:i+batch_size]

        # Forward pass
        y_pred = model(x)

        # Calculate loss
        loss = loss_fn(y_pred.view(-1, len(stoi)), y.view(-1))
        loss.backward()  # Backpropagation
        opt.step()  # Update weights
        opt.zero_grad()  # Reset gradients

    end_time = time.time()
    elapsed_time.append(end_time - start_time)

    if epoch % print_every == 0:
      torch.save(model.state_dict(), "text_model_state.pth")
    print(f'Epoch: {epoch}, Loss: {loss.item():.4f}')



Epoch: 0, Loss: 7.6888
Epoch: 1, Loss: 5.9058
Epoch: 2, Loss: 4.8807
Epoch: 3, Loss: 4.3010
Epoch: 4, Loss: 3.7695
Epoch: 5, Loss: 3.5702
Epoch: 6, Loss: 3.3308
Epoch: 7, Loss: 3.2236
Epoch: 8, Loss: 3.0898
Epoch: 9, Loss: 3.1623
Epoch: 10, Loss: 2.9162
Epoch: 11, Loss: 2.7456
Epoch: 12, Loss: 2.6050
Epoch: 13, Loss: 2.4980
Epoch: 14, Loss: 2.4227
Epoch: 15, Loss: 2.3661
Epoch: 16, Loss: 2.2806
Epoch: 17, Loss: 2.1679
Epoch: 18, Loss: 2.0811
Epoch: 19, Loss: 2.0096
Epoch: 20, Loss: 1.9510
Epoch: 21, Loss: 1.8770
Epoch: 22, Loss: 1.8091
Epoch: 23, Loss: 1.7167
Epoch: 24, Loss: 1.6487
Epoch: 25, Loss: 1.6174
Epoch: 26, Loss: 1.5702
Epoch: 27, Loss: 1.5280
Epoch: 28, Loss: 1.4779
Epoch: 29, Loss: 1.4267
Epoch: 30, Loss: 1.3599
Epoch: 31, Loss: 1.3571
Epoch: 32, Loss: 1.3062
Epoch: 33, Loss: 1.3035
Epoch: 34, Loss: 1.2622
Epoch: 35, Loss: 1.2351
Epoch: 36, Loss: 1.2117
Epoch: 37, Loss: 1.2185
Epoch: 38, Loss: 1.2039
Epoch: 39, Loss: 1.2040
Epoch: 40, Loss: 1.1864
Epoch: 41, Loss: 1.1395
Ep

In [ ]:
torch.save(model.state_dict(), "text_model_state_epoch.pth")